In [ ]:
!nvidia-smi

Fri Jan 14 20:03:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install fasttext pandas numpy tqdm simpletransformers docopt

In [4]:
!wget https://ftp.ncbi.nlm.nih.gov/pub/lu/LitCovid/biocreative/BC7-LitCovid-Train.csv
!wget https://ftp.ncbi.nlm.nih.gov/pub/lu/LitCovid/biocreative/BC7-LitCovid-Dev.csv
!git clone https://github.com/ncbi/biocreative_litcovid.git
!gdown --id 1HWZ-KC-zva75cAzqkw0cl7w1GXyoVRoP # stopwords.txt

--2022-01-14 21:05:24--  https://ftp.ncbi.nlm.nih.gov/pub/lu/LitCovid/biocreative/BC7-LitCovid-Train.csv
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.13, 165.112.9.228, 2607:f220:41f:250::229, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44081235 (42M) [text/csv]
Saving to: ‘BC7-LitCovid-Train.csv.1’

BC7-LitCovid-Train. 100%[===================>]  42.04M   110MB/s    in 0.4s    

2022-01-14 21:06:20 (110 MB/s) - ‘BC7-LitCovid-Train.csv.1’ saved [44081235/44081235]

--2022-01-14 21:06:20--  https://ftp.ncbi.nlm.nih.gov/pub/lu/LitCovid/biocreative/BC7-LitCovid-Dev.csv
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.11, 130.14.250.7, 2607:f220:41f:250::230, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11095425 (11M) [text/csv]
Saving to: ‘

In [5]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import fasttext
import time

from simpletransformers.classification import (
    MultiLabelClassificationModel, MultiLabelClassificationArgs
)

from biocreative_litcovid.biocreative_litcovid_eval import validate, print_label_based_scores, print_instance_based_scores

In [6]:
train_df = pd.read_csv('./BC7-LitCovid-Train.csv')
test_df = pd.read_csv('./BC7-LitCovid-Dev.csv')

In [7]:
train_df['content'] = train_df[['title', 'abstract']].agg(' '.join, axis=1)
test_df['content'] = test_df[['title', 'abstract']].agg(' '.join, axis=1)

In [8]:
train_df.head()

,pmid,journal,title,abstract,keywords,pub_type,authors,doi,label,content
0,32519164,J Thromb Thrombolysis,Potential role for tissue factor in the pathog...,"In December 2019, a new and highly contagious ...",covid-19;il-6;sars-cov-2;tnf-alpha;thrombosis;...,Journal Article;Review,"Bautista-Vargas, Mario;Bonilla-Abadia, Fabio;C...",10.1007/s11239-020-02172-x,Treatment;Mechanism,Potential role for tissue factor in the pathog...
1,32691006,J Tradit Complement Med,Dietary therapy and herbal medicine for COVID-...,"A novel coronavirus disease (COVID-19), transm...",covid-19;coronavirus;dietary therapy;herbal me...,Journal Article;Review,"Panyod, Suraphan;Ho, Chi-Tang;Sheen, Lee-Yan",10.1016/j.jtcme.2020.05.004,Treatment;Prevention,Dietary therapy and herbal medicine for COVID-...
2,32858315,J Affect Disord,First report of manic-like symptoms in a COVID...,"BACKGROUND: In December 2019, the novel corona...",cerebrospinal fluid;igg;manic-like symptoms;sa...,Case Reports;Journal Article,"Lu, Shaojia;Wei, Ning;Jiang, Jiajun;Wu, Lingli...",10.1016/j.jad.2020.08.031,Case Report,First report of manic-like symptoms in a COVID...
3,32985329,J Dent Res,Epidemiological Investigation of OHCWs with CO...,During the coronavirus disease 2019 (COVID-19)...,dental education;dental public health;infectio...,"Journal Article;Research Support, Non-U.S. Gov't","Meng, L;Ma, B;Cheng, Y;Bian, Z",10.1177/0022034520962087,Prevention,Epidemiological Investigation of OHCWs with CO...
4,32812051,J Antimicrob Chemother,The impact of sofosbuvir/daclatasvir or ribavi...,OBJECTIVES: Sofosbuvir and daclatasvir are dir...,NaN,Journal Article;Randomized Controlled Trial;Re...,"Eslami, Gholamali;Mousaviasl, Sajedeh;Radmanes...",10.1093/jac/dkaa331,Treatment,The impact of sofosbuvir/daclatasvir or ribavi...


In [9]:
# Preprocess
punc = '''!()-[]{};:'",<>./?@#$%^&*_~'''

stopWords = []
stopFile = open("stopwords.txt",encoding='latin-1')
lines = stopFile.readlines()
for line in lines:
    stopWords.append(line.replace("\n",""))

def preprocessFun(input):

    input = input.lower()

    for ele in input:
        if ele in punc:
            input = input.replace(ele, " ")

    ssplit = input.split()
    for word in ssplit:
        if word in stopWords:
            replaceWord = " " + word + " "
            input = input.replace(replaceWord, " ")


    return input

In [10]:
import nltk
import ssl
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

ps = PorterStemmer()

# Preprocess
punc = '''!()-[]{};:'",<>./?@#$%^&*_~'''

def preprocessFun_new(input):

    input = input.lower()

    for ele in input:
        if ele in punc:
            input = input.replace(ele, " ")

    stop_words = set(stopwords.words('english'))

    word_tokens = word_tokenize(input)

    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]

    filtered_sentence = []

    for w in word_tokens:
        if w not in stop_words:

            filtered_sentence.append(ps.stem(w))

    return " ".join(filtered_sentence)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
headers = ['PMID', 'Treatment', 'Diagnosis', 'Prevention', 'Mechanism',
              'Transmission', 'Epidemic Forecasting', 'Case Report']

def global_eval(gold_df, pred_df):
  validate(gold_df, pred_df, headers)

  print_label_based_scores(gold_df, pred_df, headers[1:])
  print_instance_based_scores(gold_df, pred_df, headers[1:])

In [12]:
random = np.random.default_rng(0)

train_data = []
validation_data = []
test_data = []

preprocess = preprocessFun_new # alternative: preprocessFun_new

for _, row in tqdm(train_df.iterrows(), desc="Transform train set into train and validation set", total=train_df.shape[0]):
    labels = row.label.split(';')
    content = preprocess(row.content)
    is_train = random.uniform() < 7/9
    if is_train:
      train_data.append([
        content,
        labels
      ])
    else:
      validation_data.append([
        content,
        labels
      ])

for _, row in tqdm(test_df.iterrows(), desc="Transform test set", total=test_df.shape[0]):
    labels = row.label.split(';')
    content = preprocess(row.content)
    test_data.append([
      content,
      labels
    ])

train = pd.DataFrame(train_data, columns=["content", "labels"])
validation = pd.DataFrame(validation_data, columns=["content", "labels"])
test = pd.DataFrame(test_data, columns=["content", "labels"])



Transform train set into train and validation set: 100%|██████████| 24960/24960 [02:24<00:00, 173.14it/s]
Transform test set: 100%|██████████| 6239/6239 [00:36<00:00, 171.85it/s]


In [13]:
train.head()

,content,labels
0,potenti role tissu factor pathogenesi hypercoa...,"[Treatment, Mechanism]"
1,dietari therapi herbal medicin covid 19 preven...,"[Treatment, Prevention]"
2,first report manic like symptom covid 19 patie...,[Case Report]
3,epidemiolog investig ohcw covid 19 coronaviru ...,[Prevention]
4,acceler launch video visit ambulatori neurolog...,[Prevention]


FastText

In [14]:
class FastTextModel:

  def df_to_fasttext(self, fp, df):
    for _, row in tqdm(df.iterrows(), desc="Transform to fastText format", total=df.shape[0]):
      labels = []
      for label in row.labels:
        labels.append(f"__label__{label.replace(' ', '-')}")
      doc = f"{' '.join(labels)} {row.content}\n"
      fp.write(doc)

  def __init__(self, train, validation):
    self.model = None
    with open("train.txt", "w") as train_f, open("valid.txt", "w") as valid_f:
      self.df_to_fasttext(train_f, train)
      self.df_to_fasttext(valid_f, validation)
    
  def train(self, save=False):
    model = fasttext.train_supervised("train.txt", 
                                      autotuneValidationFile="valid.txt",
                                      autotuneDuration=3*60)

    if save:
      model.save_model(f"cmpe493-termproject-{int(time.time())}.ftz")
    self.model = model

  def predict(self, content):
    scores = {}
    res = self.model.predict(preprocess(content), k=-1)
    for label, score in zip(res[0], res[1]):
      scores[label.split('__label__')[1].replace('-', ' ')] = score
    return scores

  def eval(self, test_df, save_path=None):
    test_results_df = pd.DataFrame()
    gold_df = pd.DataFrame()

    for index, row in tqdm(test_df.iterrows(), desc="Predict Test Set with Trained Model", total=test_df.shape[0]):
      scores = self.predict(row.content)
      # print(scores)
      for label in headers[1:]:
        test_results_df.loc[index, 'PMID'] = int(row.pmid)
        test_results_df.loc[index, label] = scores[label] if label in scores else 0

        gold_df.loc[index, 'PMID'] = int(row.pmid)
        gold_df.loc[index, label] = 1 if label in row.label.split(';') else 0
      
    test_results_df.PMID = test_results_df.PMID.astype(int)
    gold_df.PMID = gold_df.PMID.astype(int)
  
    if save_path:
        test_results_df.to_csv(save_path, index=False, header=True)

    print(gold_df.head())
    print(test_results_df.head())

    global_eval(gold_df, test_results_df)

In [15]:
ftModel = FastTextModel(train, validation)

Transform to fastText format: 100%|██████████| 5523/5523 [00:00<00:00, 8258.39it/s]


In [16]:
ftModel.train()

Progress:  53.1% Trials:    4 Best score:  0.781260 ETA:   0h 1m24stcmalloc: large alloc 1185284096 bytes == 0x5583f4326000 @  0x7fd261477887 0x7fd237cf95cd 0x7fd237cde5de 0x7fd237ce41c5 0x7fd237cf45e2 0x7fd237c95e1f 0x7fd237cbfdd7 0x5583c00744b0 0x5583c0074240 0x5583c00e80f3 0x5583c00e29ee 0x5583c0075bda 0x5583c00e4737 0x5583c00e29ee 0x5583c0075bda 0x5583c00e3c0d 0x5583c00e29ee 0x5583c00e26f3 0x5583c00e0b60 0x5583c0074349 0x5583c0074240 0x5583c00e7973 0x5583c0167dd5 0x5583c00e4c6f 0x5583c0167dd5 0x5583c00e4c6f 0x5583c0167dd5 0x5583c00742ed 0x5583c0165e1d 0x5583c00e7e99 0x5583c0075afa
Progress: 100.0% Trials:    5 Best score:  0.781260 ETA:   0h 0m 0s
Training again with best arguments
Read 2M words
Number of words:  35592
Number of labels: 7
Progress: 100.0% words/sec/thread: 4625908 lr:  0.000000 avg.loss:  0.530046 ETA:   0h 0m 0s


In [17]:
ftModel.eval(test_df, save_path="./fasttext-results.csv")

Predict Test Set with Trained Model: 100%|██████████| 6239/6239 [01:09<00:00, 89.70it/s]


       PMID  Treatment  ...  Epidemic Forecasting  Case Report
0  32653511        1.0  ...                   0.0          0.0
1  32427104        0.0  ...                   1.0          0.0
2  32605414        0.0  ...                   0.0          0.0
3  33066278        1.0  ...                   0.0          0.0
4  32303609        1.0  ...                   0.0          0.0

[5 rows x 8 columns]
       PMID  Treatment  ...  Epidemic Forecasting  Case Report
0  32653511   0.812155  ...              0.000106     0.014511
1  32427104   0.000702  ...              0.140286     0.000012
2  32605414   0.000457  ...              0.000773     0.000025
3  33066278   0.749120  ...              0.000179     0.000418
4  32303609   0.540931  ...              0.000507     0.001189

[5 rows x 8 columns]
validation starts...
validation passes...
label-based measures...
                      precision    recall  f1-score   support

           Treatment     0.9502    0.5356    0.6850      2207
         

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


instance-based measures
mean precision 0.8739
mean recall 0.7338
f1 0.7977


In [ ]:
# train_label_int = train
# train_label_int.labels = train.labels.map(lambda labels: [headers.index(l) for l in labels])

# train_label_int

train_st_data = []

for index, row in tqdm(train.iterrows(), desc="-", total=train.shape[0]):
  labels = []
  # print(headers)
  # print(row.labels)
  for label in headers[1:]:
    labels.append( 1 if label in row.labels else 0 )
  # print(labels)

  train_st_data.append([
    row.content,
    labels
  ])

train_st = pd.DataFrame(train_st_data)
train_st.columns = ["text", "labels"]
train_st

-: 100%|██████████| 19437/19437 [00:02<00:00, 7144.93it/s]


,text,labels
0,potenti role tissu factor pathogenesi hypercoa...,"[1, 0, 0, 1, 0, 0, 0]"
1,dietari therapi herbal medicin covid 19 preven...,"[1, 0, 1, 0, 0, 0, 0]"
2,first report manic like symptom covid 19 patie...,"[0, 0, 0, 0, 0, 0, 1]"
3,epidemiolog investig ohcw covid 19 coronaviru ...,"[0, 0, 1, 0, 0, 0, 0]"
4,acceler launch video visit ambulatori neurolog...,"[0, 0, 1, 0, 0, 0, 0]"
...,...,...
19432,predict factor diseas progress hospit patient ...,"[1, 1, 0, 0, 0, 0, 0]"
19433,covid 19 outbreak itali clinic radiolog presen...,"[0, 0, 0, 0, 0, 0, 1]"
19434,determin diseas sever covid 19 patient use dee...,"[0, 1, 0, 0, 0, 0, 0]"
19435,baselin electrolyt abnorm would relat poor pro...,"[1, 1, 0, 0, 0, 0, 0]"


In [ ]:
valid_st_data = []

for index, row in tqdm(validation.iterrows(), desc="-", total=validation.shape[0]):
  labels = []
  # print(headers)
  # print(row.labels)
  for label in headers[1:]:
    labels.append( 1 if label in row.labels else 0 )
  # print(labels)

  valid_st_data.append([
    row.content,
    labels
  ])

valid_st = pd.DataFrame(valid_st_data)
valid_st.columns = ["text", "labels"]
valid_st

-: 100%|██████████| 5523/5523 [00:00<00:00, 8180.75it/s]


,text,labels
0,impact sofosbuvir daclatasvir ribavirin patien...,"[1, 0, 0, 0, 0, 0, 0]"
1,coronaviru diseas 2019 pandem effect critic ca...,"[0, 0, 1, 0, 0, 0, 0]"
2,coronaviru diseas 2019 covid 19 experi protoco...,"[0, 0, 1, 0, 0, 0, 0]"
3,chest ct find asymptomat case covid 19 systema...,"[0, 1, 0, 0, 0, 0, 0]"
4,profil natur anticoagul coagul factor anti pho...,"[1, 1, 0, 0, 0, 0, 0]"
...,...,...
5518,impact consequ sar cov 2 pandem singl univers ...,"[0, 0, 1, 0, 0, 0, 0]"
5519,procalcitonin secondari bacteri infect covid 1...,"[1, 1, 0, 0, 0, 0, 0]"
5520,neutrophil count albumin ratio new predictor m...,"[1, 1, 0, 0, 0, 0, 0]"
5521,medic recommend home confin footbal train covi...,"[0, 0, 1, 0, 0, 0, 0]"


In [ ]:
import torch


cuda_available = torch.cuda.is_available()

cuda_available

True

In [ ]:
class DeepModel:
    
    def __init__(self, model_type, model_name, epoch, load_path=None):
        self.model_args = MultiLabelClassificationArgs(
            num_train_epochs=epoch,
            overwrite_output_dir=True,
        )

        self.model = MultiLabelClassificationModel(
            model_type,
            load_path if load_path else model_name,
            num_labels=7,
            args=self.model_args,
            use_cuda=torch.cuda.is_available(),
        )
    
    def train(self, train_df, eval_df=None, save_path=None):
        self.model.train_model(train_df, eval_df=eval_df)
        if save_path:
            temp_model_path = './outputs/'

            if not os.path.exists(save_path):
                os.makedirs(save_path)

            for f in listdir(temp_model_path):
                if isfile(join(temp_model_path, f)):
                    file_path = join(temp_model_path, f)
                    shutil.copy2(file_path, save_path)
            print(f"Model trained and saved into {save_path}")
        
    def evaluate(self, test_df, save_path=None):
        scores = self.model.predict(test_df.content.map(lambda x: preprocessFun(x)).tolist())
        
        test_results_df = pd.DataFrame()
        gold_df = pd.DataFrame()

        for index, row in tqdm(test_df.iterrows(), desc="Predict Test Set with Trained Model", total=test_df.shape[0]):
            for label in headers[1:]:
                test_results_df.loc[index, 'PMID'] = int(row.pmid)
                test_results_df.loc[index, label] = scores[1][index][headers.index(label)-1]

                gold_df.loc[index, 'PMID'] = int(row.pmid)
                gold_df.loc[index, label] = 1 if label in row.label.split(';') else 0

        test_results_df.PMID = test_results_df.PMID.astype(int)
        gold_df.PMID = gold_df.PMID.astype(int)

        if save_path:
            test_results_df.to_csv(save_path, index=False, header=True)

        global_eval(gold_df, test_results_df)

In [ ]:

# Trained Models
!gdown --id 17cwDrqwoa3tM4cO8lNOt2DxzWH0ORdp4
!gdown --id 19YdtKbwWNsP-5VrWcFQvQ14NflUpT6-o
!gdown --id 18uHAIFfHfPAEyIHGeblxfa8Iy79wyu_3
!gdown --id 1zJ07CghwloHAqRwijjWuDoQLmkMGbe1m

!unzip biobert-5-npp.zip
!unzip scibert-5-npp.zip
!unzip biobert-5-pp.zip
!unzip scibert-5-pp.zip

Archive:  biobert-5-npp.zip
  inflating: biobert-5epoch-npp/config.json  
  inflating: biobert-5epoch-npp/model_args.json  
  inflating: biobert-5epoch-npp/pytorch_model.bin  
  inflating: biobert-5epoch-npp/special_tokens_map.json  
  inflating: biobert-5epoch-npp/tokenizer_config.json  
  inflating: biobert-5epoch-npp/training_args.bin  
  inflating: biobert-5epoch-npp/vocab.txt  
Archive:  scibert-5-npp.zip
  inflating: scibert-5epoch-npp/config.json  
  inflating: scibert-5epoch-npp/model_args.json  
  inflating: scibert-5epoch-npp/pytorch_model.bin  
  inflating: scibert-5epoch-npp/special_tokens_map.json  
  inflating: scibert-5epoch-npp/tokenizer_config.json  
  inflating: scibert-5epoch-npp/training_args.bin  
  inflating: scibert-5epoch-npp/vocab.txt  


In [ ]:
biobert = DeepModel(
    "bert",
    "dmis-lab/biobert-v1.1",
    5,
    load_path="./biobert-5epoch-npp"
)
# biobert.train(train_st, eval_df=valid_st, save_path="./biobert-5epoch-pp/")
biobert.evaluate(test_df, save_path="./biobert-5-npp-results.csv")

In [ ]:
scibert = DeepModel(
    "bert",
    "allenai/scibert_scivocab_uncased",
    5,
    load_path="scibert-5epoch-npp"
)
# scibert.train(train_st, eval_df=valid_st, save_path="./scibert-5epoch-pp/")
scibert.evaluate(test_df, save_path="./scibert-5-npp-results.csv")